<a href="https://colab.research.google.com/github/PouriaRouzrokh/AI-Deep-Learning-Lab-2025/blob/main/DL%20Lab%20-%20An%20Introduction%20to%20Multi-agent%20Artificial%20Intelligence%20Systems%20in%20Radiology/RSNA2025_DLL_AgentiAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**An Introduction to Multi-agent Artificial Intelligence Systems in Radiology**


- **Moderator:** Pouria Rouzrokh MD MPH MHPE [[1]](https://www.linkedin.com/in/pouria-rouzrokh/)
- **Speaker:** Moein Shariatnia MD [[2]](https://www.linkedin.com/in/melina-hosseiny-m-d-0a12b7a1)  
- **Speaker:** Melina Hosseiny MD [[3]](https://www.linkedin.com/in/moein-shariatnia?originalSubdomain=ir)

## Overview

Welcome to the Deep Learning Labs at RSNA2025!

In this hands-on workshop, we will explore the concept of AI agents, emphasizing their importance and unique advantages over single, monolithic language models.

## What You'll Learn

This notebook provides a quick introduction to building agentic and multi-agent AI systems using **Google's Agent Development Kit (ADK)**. Using this framework, we'll explore:

1. **Basic Agents**: Simple LLM-based agents without tools
2. **Augmented Agents**: Agents enhanced with tools (e.g., Google Search)
3. **Multi-Agent Pipelines**: Sequential, parallel, and loop architectures
4. **Advanced Tools and Pipelines**: How to equip our agents to MCP tools, run them independantly, and many more things!

And at the very end of this notebook, we will also look at a more production-ready product built with the Google ADK framework so that you can see the actual power of AI agents; ***so stay tuned!***

---
*P.S. Throughout this notebook, the term “agent” is used broadly to describe many types of chatbots, including those that are essentially LLMs or augmented LLMs rather than true agents. This aligns with the general usage of the term “agent” within the Google ADK.*

## Setup

Let's start by downloading the data that we need and setting up the scene!

In [ ]:
# @title
import os
import subprocess
import shutil
from urllib.parse import unquote

def clone_specific_folder(repo_and_path, destination=None):
    repo_and_path = unquote(repo_and_path)

    parts = repo_and_path.split("/tree/")
    base = parts[0].replace("https://github.com/", "")
    repo_url = f"https://github.com/{base}.git"
    branch_and_path = parts[1].split("/", 1)
    branch = branch_and_path[0]
    folder_path = branch_and_path[1] if len(branch_and_path) > 1 else ""

    folder_path = folder_path.rstrip("/")
    destination = destination or os.path.basename(folder_path) or "cloned_repo"

    os.makedirs(destination, exist_ok=True)
    cwd = os.getcwd()
    os.chdir(destination)

    subprocess.run(["git", "init"], capture_output=True)
    subprocess.run(["git", "remote", "add", "origin", repo_url], capture_output=True)
    subprocess.run(["git", "config", "core.sparseCheckout", "true"], capture_output=True)

    with open(".git/info/sparse-checkout", "w") as f:
        f.write(f"{folder_path}/*\n" if folder_path else "*\n")

    try:
        subprocess.run(["git", "pull", "origin", branch], check=True, capture_output=True)
    except:
        subprocess.run(["git", "pull", "origin", "master"], check=True, capture_output=True)

    if folder_path and os.path.exists(folder_path):
        shutil.move(folder_path, "_temp")
        for item in os.listdir("."):
            if item not in ["_temp", ".git"]:
                shutil.rmtree(item) if os.path.isdir(item) else os.remove(item)
        for item in os.listdir("_temp"):
            shutil.move(f"_temp/{item}", ".")
        shutil.rmtree("_temp")

    os.chdir(cwd)

In [ ]:
# Ignore the warnings (Google ADK has lots of warnings!)
import warnings
warnings.filterwarnings("ignore")

# Clone the deep learning lab data from the RSNA2025 Deep Learning Labs repository.
target_link = "https://github.com/RSNA/AI-Deep-Learning-Lab-2025/tree/main/DL%20Lab%20-%20An%20Introduction%20to%20Multi-agent%20Artificial%20Intelligence%20Systems%20in%20Radiology/scripts/agents"
clone_specific_folder(target_link)

# Remove the sample_data folder
shutil.rmtree("sample_data", ignore_errors=True)


### Getting Your Google API Key

The Google ADK framework works with the Gemini family of large language models by default (it can also work with other models like OpenAI, Claude, etc.). To run this notebook smoothly, you'll need an API key from Google. Google provides everyone with a free API key that includes a generous free tier, and even if you exceed the free tier, the costs for these language models are negligible.

To get your API key, open the following link (if you're attending the workshop in person, follow along on the screen):

[https://aistudio.google.com/app/api-keys](https://aistudio.google.com/app/api-keys)

After you get your API key, go ahead and enter it in the cell below, which will securely store it for use in this notebook.

In [ ]:
import os
from getpass import getpass

# Securely prompt for API key
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable
os.environ['GOOGLE_API_KEY'] = api_key

print("✓ API key has been set successfully!")

Enter your Google API Key: ··········
✓ API key has been set successfully!


Last but not least, we need to install the packages that we need for our today's workshop.

In [ ]:
%pip install -q google-adk python-dotenv requests pydantic

Alright, now we are ready to start talking about agents!

## **Part 1. An Introduction to AI Agents**

> Associated Agent: **basic_radiology_assistant**

At its core, an **agent** is a Large Language Model (LLM) enhanced with the ability to perform tasks and make decisions in a semi-automated way. Unlike a plain LLM that only answers a single query, an agent can:

- Receive user input
- Process it using both its pre-trained knowledge and, if available, predefined external tools
- Make decisions step-by-step, continuing its work toward a goal, reaching checkpoints, or halting if guardrails or stopping criteria are triggered
- Return an answer or outcome—not just in response to a question, but potentially after a sequence of actions or automated reasoning steps

A basic agent without any tools behaves just like an LLM (using only its internal knowledge), but more advanced agents can reason, use external tools, and act with degrees of autonomy.


**1\. Writing your first agent with Google ADK**

To begin exploring the world of agents, let’s first build a dummy radiology assistant agent that can answer any questions you have about radiology. Despite this mouthful of a title, this “agent” is, in fact, just a large language model (such as Gemini 2.5 Flash in our case) that we instruct to only answer radiology-related questions. It will not have any tools and will not be able to do anything fancy apart from continuing the conversation as much as needed, which is basically something every large language model can do. They are all quite verbose, after all :)

Take a look at the code below to see how we can implement this using the Google ADK.

```python
from google.adk.agents import Agent

# Define the basic radiology assistant agent
basic_radiology_assistant = Agent(
    name="basic_radiology_assistant",
    model="gemini-2.5-flash",  # Using Gemini Flash model
    instruction=(
        "You are a helpful assistant specialized in radiology. "
        "Your primary role is to answer questions about radiology, including:\n"
        "- Medical imaging techniques and modalities\n"
        "- Radiological findings and interpretations\n"
        "- Anatomy and pathology relevant to radiology\n"
        "- Imaging protocols and best practices\n"
        "- Radiological terminology and concepts\n\n"
        "You should politely decline to answer questions that are not related to radiology. "
        "When asked about non-radiology topics, kindly redirect the conversation back to "
        "radiology-related questions."
    ),
    description=(
        "A basic specialized assistant that answers questions about radiology and "
        "avoids answering miscellaneous questions. No web search capabilities."
    ),
)
```

As you can see, the agent receives a few arguments, the most important of which is its instructions. In addition to instructions, you might notice that the agent also takes a description, a name, and, obviously, a model. The model is essentially the “brain” of the agent, and we can pass any of the Gemini models to Google ADK agents. However, it is important to note that we can also pass other families of models, such as those from OpenAI and Anthropic, to Google ADK as well. This is very straightforward, and you can read more about it in the documentation, but we will not delve into those details in this notebook.

You may wonder why this agent needs both a name and a description. After all, the instructions alone would suffice for this simple, single-agent task. As we move forward and our agent systems become more complex, however, each agent will increasingly need an identity. Agents need to understand each other’s “philosophy of being,” main responsibilities, and reason for existing. The name and description of an agent are primarily used so that other agents can refer to it and understand its role. Since we only have a single agent here, this will become more important later when we introduce multi-agent setups.

---
**2\. Executing your first agent**

Well, now that we have coded our very first simple agent, you might be waiting for us to show you the syntax for executing this agent from this current notebook. However, when it comes to **Google ADK**, you should bear in mind that this package has been built with ultimate prioritization of deployment.

But being deployment-ready does not necessarily mean that it is user-friendly—or at least notebook-friendly in our case. In fact, running Google ADK agents is most of the time easier from within the **terminal** when they are defined inside Python scripts rather than from within a Jupyter or Google Colab environment.

Of course, there are ways for running the agents in notebook environments, but those might be a little more complicated for beginners and we will not get into those approaches until the very last agent that we will deploy together.

---

So what other ways do we have for running them, you might ask? Well, easy! Thankfully, we can still run CLI commands from Google Colab through by adding a "!" to the start of a code cell line. This will tell Colab that the commands need to be interpreted from shell. Now that we know this trick, we can use all our agents by running a command like the following: `adk run agents/basic_radiology_assistant`.

This will open a very basic chat interface with the agent in terminal, which will still work but might not be that beautiful. The following cell will implement a function to Display our conversation with the agents in a more decent way. Obviously, this function is entirely optional and its content is not relevant to our workshop at all, so feel free to ignore it if you don't feel like reading it.

In [ ]:
# @title Optional code to format the appearance of chat conversation with agents.

# 1. Ensure markdown library is present
import sys
import subprocess
try:
    import markdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "markdown"])
    import markdown

import pexpect
import os
import re
import time
from IPython.display import display, HTML, clear_output

def chat_with_agent_clean(agent_path, session_id=None):
    # --- CONFIGURATION ---
    TITLES = {
        "part1": "Part 1: Basic Radiology Assistant",
        "part2": "Part 2: Web Search Assistant",
        "part3": "Part 3: Biography Agent",
        "part4": "Part 4: Biography Agent (Fixed)",
        "part5": "Part 5: Networking Agent",
        "part6": "Part 6: Radiology Researcher Agent"
    }

    match = re.search(r'(part\d+)', agent_path, re.IGNORECASE)
    display_title = TITLES.get(match.group(1).lower()) if match and match.group(1).lower() in TITLES else os.path.basename(agent_path).replace('_', ' ').title()

    session_html = f'<span style="color: #d9534f; font-weight: bold; margin-left: 10px;">🔴 Recording Session: {session_id}</span>' if session_id else '<span style="color: #888;">Interactive ADK Session</span>'

    chat_history = []

    # --- RENDER ENGINE ---
    def render_chat(history, status=None):
        clear_output(wait=True)

        css = '''
        <style>
            .chat-container { max-width: 700px; margin: 0 auto; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif; }
            .agent-row { display: flex; margin-bottom: 20px; align-items: flex-start; }
            .user-row { display: flex; margin-bottom: 20px; justify-content: flex-end; align-items: flex-end; }
            .avatar { width: 40px; height: 40px; border-radius: 50%; display: flex; align-items: center; justify-content: center; font-size: 22px; flex-shrink: 0; box-shadow: 0 2px 3px rgba(0,0,0,0.1); }
            .avatar-agent { background: #f0f0f0; margin-right: 15px; margin-top: 5px; }
            .avatar-user { background: #e3f2fd; margin-left: 15px; }
            .agent-bubble { background-color: #f8f9fa; color: #1a1a1a; padding: 15px 22px; border-radius: 20px; border-top-left-radius: 4px; max-width: 80%; line-height: 1.6; font-size: 16px; border: 1px solid #eee; box-shadow: 0 1px 2px rgba(0,0,0,0.05); }
            .user-bubble { background-color: #007bff; color: white; padding: 15px 22px; border-radius: 20px; border-bottom-right-radius: 4px; max-width: 80%; line-height: 1.6; font-size: 16px; text-align: left; box-shadow: 0 1px 2px rgba(0,0,0,0.1); }

            /* Markdown Styles */
            .agent-bubble p { margin: 0 0 10px 0; } .agent-bubble p:last-child { margin: 0; }
            .agent-bubble ul, .agent-bubble ol { margin: 5px 0 10px 20px; padding: 0; }
            .agent-bubble li { margin-bottom: 5px; }
            .agent-bubble strong { font-weight: 700; color: #000; }
            .agent-bubble pre { background: #2d2d2d; color: #ccc; padding: 10px; border-radius: 8px; overflow-x: auto; }
            .agent-bubble code { background: #eee; padding: 2px 5px; border-radius: 3px; font-family: monospace; font-size: 0.9em; }
            .agent-bubble pre code { background: transparent; padding: 0; color: #f8f8f2; }

            .status-indicator { font-size: 13px; color: #888; text-align: center; margin-top: 20px; font-style: italic; animation: pulse 1.5s infinite; }
            @keyframes pulse { 0% { opacity: 0.5; } 50% { opacity: 1; } 100% { opacity: 0.5; } }
            .sys-log { font-family: monospace; font-size: 11px; color: #aaa; margin: 5px 0; text-align: center; }
        </style>
        '''

        html = f'''{css}
        <div class="chat-container">
            <h2 style="color: #333; margin-bottom: 5px; text-align: center;">🩺 {display_title}</h2>
            <p style="font-size: 13px; margin-top: 0; text-align: center;">{session_html}</p>
            <hr style="border: 0; border-top: 1px solid #eee; margin-bottom: 30px;">
        '''

        for msg in history:
            if msg['role'] == 'sys':
                html += f'<div class="sys-log">{msg["content"]}</div>'
            elif msg['role'] == 'agent':
                # MODIFIED: Added extensions=['nl2br', 'fenced_code']
                # nl2br: Converts newlines to <br> tags
                # fenced_code: Allows proper rendering of ```code blocks```
                formatted_text = markdown.markdown(msg['content'], extensions=['nl2br', 'fenced_code'])
                html += f'<div class="agent-row"><div class="avatar avatar-agent">🤖</div><div class="agent-bubble">{formatted_text}</div></div>'
            elif msg['role'] == 'user':
                html += f'<div class="user-row"><div class="user-bubble">{msg["content"]}</div><div class="avatar avatar-user">👤</div></div>'

        if status:
            html += f'<div class="status-indicator">{status}</div>'

        html += '</div>'
        display(HTML(html))

    # --- START PROCESS ---
    cmd = f"adk run {agent_path}"
    if session_id:
        cmd += f" --save_session --session_id {session_id}"

    env = os.environ.copy()
    env["PYTHONWARNINGS"] = "ignore"

    child = pexpect.spawn(cmd, env=env, encoding='utf-8', timeout=None)
    time.sleep(0.5)

    try: child.setecho(False)
    except: pass

    last_user_input = ""

    render_chat(chat_history, status="🟢 Starting Agent...")

    try:
        while True:
            index = child.expect(['\[user\]:', pexpect.EOF])

            if index == 0: # Prompt found
                raw = child.before.strip()

                if last_user_input and raw.startswith(last_user_input):
                    raw = raw[len(last_user_input):].strip()

                agent_lines = []
                for line in raw.split('\n'):
                    line = line.strip()
                    if not line: continue
                    if any(x in line for x in ["Log setup", "Running agent", "To access latest", "Type exit"]):
                        chat_history.append({'role': 'sys', 'content': line})
                    else:
                        clean = re.sub(r'^\[.*?\]:\s*', '', line)
                        if clean: agent_lines.append(clean)

                if agent_lines:
                    full_response = "\n".join(agent_lines)
                    chat_history.append({'role': 'agent', 'content': full_response})

                # Show chat without status (Input box will appear below)
                render_chat(chat_history, status=None)
                time.sleep(0.1)

                try:
                    user_input = input("Type your input here: ")
                except KeyboardInterrupt:
                    print("\nStopped.")
                    break

                chat_history.append({'role': 'user', 'content': user_input})
                last_user_input = user_input

                # Show "Agent is replying..." and hide input
                render_chat(chat_history, status="🟢 Agent is replying...")

                child.send(user_input + "\n")

            elif index == 1: # EOF
                chat_history.append({'role': 'sys', 'content': "🔴 Session ended."})
                render_chat(chat_history)
                break

    except KeyboardInterrupt:
        child.close()
        print("Chat closed.")

Now that we have this function, we can go ahead and call our first agent. In order to call an agent using the `adk run` command, we need to pass the path to the parent directory of that agent to this command. Run the following cell and start talking with our very first agent!

In [ ]:
# Run the Part 1 agent|

chat_with_agent_clean('agents/part1/basic_radiology_assistant', session_id="123")

**3\. Exploring the Session Details**

Great! Did you have fun with this first agent? Before we proceed forward, Did you notice that we actually also passed a session ID to our chat function? This is because the `adk run` command optionally accepts a `session ID` argument and when you pass that, It logs down all the details of the conversation in a JSON file.

Pause for a second and take a look at The JSON file which was saved inside the basic radiology assistant agent folder. There are many keys and values in this file. What do you think this data is about?

Well, actually this JSON file shows the great amount of **logging** that the Google ADK framework does behind the scenes when you execute an agent. For example:

- **Events** are referring to anything that might have happened since the time that you started your agent—any kind of task that you assigned to that agent, what that agent replied to, how many tokens it used, what kind of modality your input and output data had
- **Artifacts** might include images, videos, or voice commands that needed to be stored
- **Memory** tracks variables that the agent should have kept in its memory
- **Sessions** are individual instances of agent activity. Agents in Google ADK can engage in **multiple simultaneous conversations** with different users (or take actions on their behalves), where each user has its own session. Therefore, keeping track of which sessions are currently active, who has spoken with the agent, and how events and states have evolved in each session is another fundamental aspect of the Google ADK framework that makes it an ideal choice for web service applications that can later be served to **FastAPI** or other backend-to-framework communication protocols.

## **Part 2. Augmented Language Models**

> Associated Agent: **web_search_agent**

Even though we called our dummy radiology assistant an "agent," in reality it was just a simple, naive large language model. Well, let's improve it a little bit!

What we can do with Google ADK is to equip our agents *(note that here we are using the term "agent" a little bit loosely)* with different **tools**.

---

### What Are Tools?

Tools can be defined very broadly, but in simple terms, a **tool** is a programming interface that allows the agent to achieve something that it could not achieve—or could not perform well—on its own.

For example, Google ADK itself has a very useful built-in tool that almost all agents might benefit from, and that is **Google Search**. Well, at the end of the day, ADK belongs to the company owning our beloved Google Search! So you can easily equip any agent with a Google Search toolbox that it can use to programmatically search for whatever it needs to satisfy you as the user commanding that agent.

---

### Other Built-in Tools

Google has some other built-in tools as well, the most important of which (in addition to Google Search) is **Code Execution**, which enables an agent to write and execute programming code. Even though this tool is also very important, we will not talk about it in this notebook and instead stick to the Google Search tool.

---

In the following code snippet, you can see the code for a simple web search agent that has different instructions compared to the prior agent we coded, and this time is equipped with the Google Search tool imported from Google ADK.RetryClaude can make mistakes. Please double-check responses.

```python
from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="web_search_agent",
    model="gemini-2.5-flash",  # Using Gemini Flash model
    instruction=(
        "You are a specialized radiology assistant that uses web search to answer questions. "
        "Your primary role is to answer questions about radiology by performing web searches, including:\n"
        "- Medical imaging techniques and modalities\n"
        "- Radiological findings and interpretations\n"
        "- Anatomy and pathology relevant to radiology\n"
        "- Imaging protocols and best practices\n"
        "- Radiological terminology and concepts\n"
        "- Recent research and developments in radiology\n"
        "- Clinical guidelines and recommendations\n\n"
        "When answering radiology questions:\n"
        "- Use the google_search tool to find up-to-date and accurate information\n"
        "- Synthesize information from multiple sources when available\n"
        "- Provide clear, accurate, and well-sourced answers\n"
        "- Cite relevant sources when appropriate\n\n"
        "You should politely decline to answer questions that are not related to radiology. "
        "When asked about non-radiology topics, kindly redirect the conversation back to "
        "radiology-related questions and explain that you specialize only in radiology topics."
    ),
    description=(
        "A specialized radiology assistant that answers questions by performing web searches. "
        "Uses Google Search to provide up-to-date information about radiology topics and "
        "avoids answering questions unrelated to radiology."
    ),
    tools=[google_search],  # Equipped with Google Search tool
)
```


Run the following cell to interact with the above agent. Try to challenge this agent to ensure it is answering you by grounding in facts retrieved from internet. For example, you can ask it about recent events and things that the Gemini model might not have known until very recently.  

In [ ]:
# Run the Part 2 agent

chat_with_agent_clean('agents/part2/web_search_agent', session_id="123")

## **Part 3. Custom Tools and ADK Limitations**

> Associated Agent: **biogeraphy_agent_broken**

All right, now that we saw our very first example of a large language model using tools (AKA an augmented language model), let's see if we can create our own custom tools.

Obviously, the answer to this question is positive! In fact, Google ADK and many other agentic frameworks allow you to put together programming functions and simply pass them as tools to an agent.

---

### Tool Conventions Across Frameworks

Now, depending on the agentic framework you are using, you might need to follow different conventions for ensuring that your function tools are compatible with the agents you are using. But almost all of these conventions are minimal and very easy to handle.

One of the conventions that almost all agentic frameworks adhere to is to ensure that your custom functions have **very good documentation**. In Python, for example, good documentation means having a very good **docstring** and **type hinting** before anything else.

---

### Why Documentation Matters for Agents

You might wonder why this is necessary. Well, even though good type hinting and docstrings are always recommended to enhance code readability, for agents, it is of **utmost importance**.

When an agent wants to identify the right tool among the many tools that it might have for a specific task, the agent cannot read the entire length of the function. Instead, agents go through the **docstring** and **input/output arguments** of a function and try to understand what purpose that function is serving. Then they use it in the right context.

---

### Building a Biography Agent

Enough talking—let's go ahead and work on our third agent, which is a **biography agent**.

In this agent, we want to:
1. Look for a person's biography by searching Google for their name
2. Write down the biography we found as a markdown file

Even though the agent itself might be able to write very good markdown for us, we can still rely on an external tool to take the agent's input and write it as a markdown file.

The following code snippets show how such a tool can be created and then how we can pass it to an agent.

```python
from pathlib import Path

def write_biography_markdown(biography_content: str, person_name: str, filename: str = None) -> str:
    """Write a biography to a markdown file.
    
    This function takes biography content and saves it to a markdown file.
    The file will be saved in the same directory as the agent file.
    
    Args:
        biography_content: The biography content in markdown format.
        person_name: The name of the person the biography is about.
        filename: Optional custom filename. If not provided, uses person_name_biography.md
    
    Returns:
        A string message indicating success and the file path where the markdown was saved.
    
    Example:
        write_biography_markdown("# John Doe\n\nJohn Doe was born...", "John Doe")
    """
    try:
        # Get the directory where this agent file is located
        agent_dir = Path(__file__).parent
        
        # Generate filename if not provided
        if filename is None:
            # Sanitize person name for filename
            safe_name = "".join(c if c.isalnum() or c in (' ', '-', '_') else '' for c in person_name)
            safe_name = safe_name.replace(' ', '_').lower()
            filename = f"{safe_name}_biography.md"
        
        # Ensure .md extension
        if not filename.endswith('.md'):
            filename = f"{filename}.md"
        
        markdown_path = agent_dir / filename
        
        # Write markdown file
        with open(markdown_path, 'w', encoding='utf-8') as f:
            f.write(biography_content)
        
        return (
            f"Successfully created biography markdown file for {person_name}. "
            f"File saved at: {markdown_path.absolute()}"
        )
    
    except Exception as e:
        return f"Error creating markdown file: {str(e)}"
```

And here is our agent:

```python
root_agent = Agent(
    name="biography_agent_broken",
    model="gemini-2.5-flash",
    instruction=(
        "You are a biography writer agent. Your role is to research people and write their biographies.\n\n"
        
        "When given a person's name:\n"
        "1. Use the google_search tool to search for information about the person.\n"
        "2. Gather comprehensive information including:\n"
        "   - Early life and background\n"
        "   - Education\n"
        "   - Career and achievements\n"
        "   - Notable works or contributions\n"
        "   - Personal life (if relevant and publicly available)\n"
        "   - Awards and recognition\n"
        "   - Current status or legacy\n\n"
        "3. Synthesize the information into a well-structured biography in markdown format.\n"
        "4. Use the write_biography_markdown function to save the biography to a markdown file.\n\n"
        
        "The biography should be:\n"
        "- Well-organized with clear sections\n"
        "- Factual and accurate\n"
        "- Properly formatted in markdown\n"
        "- Include sources when possible\n"
        "- Professional and engaging\n\n"
        
        "Example markdown structure:\n"
        "# [Person's Name]\n\n"
        "## Early Life\n\n"
        "## Education\n\n"
        "## Career\n\n"
        "## Achievements\n\n"
        "## Personal Life\n\n"
        "## Legacy\n\n"
    ),
    description=(
        "A biography writer that searches the internet for information about people "
        "and writes their biographies in markdown files. "
        "BROKEN: This agent mixes Google Search tool with custom function tools in a single agent."
    ),
    tools=[
        google_search,  # Google Search tool
        tools.write_biography_markdown,  # Custom markdown writing function
    ],
)

```

Great, now let's go ahead and execute it!

In [ ]:
# Run the Part 3 agent

chat_with_agent_clean('agents/part3/biography_agent_broken')

### Understanding Agent Failures and Framework Limitations

Well, what happened? Why did our agent not work and our session ended prematurely?

As you can imagine from the title of this part (Agent Failure), the failure of this agent was not unexpected. Actually, we **purposefully ignored a bug** in how we set up our agent above to teach you something about the Google ADK framework.

---

### Framework Restrictions

Like almost every other framework that deals with language models, Google ADK comes with lots of restrictions. And because ADK targets deployment and production readiness, some of these restrictions are very strictly enforced for developers.

For example, one of the interesting restrictions that this package has is that its **native tools** (such as Google Search or Code Execution) **cannot coexist with custom tools** that the user has developed and wants the agent to use.

This means that when we were trying to pass:
```python
tools=[
    google_search,  # Google Search tool
    tools.write_biography_markdown,  # Custom markdown writing function
],
```

we violated one of the limitations of Google ADK. These two tools cannot be passed at the same time to an agent.

---

### Why These Restrictions Exist

Now you might wonder why this is the case. Honestly, we don't have a clear answer for you, but you are more than welcome to dig deeper into this—maybe submit an issue to the GitHub page of Google ADK or try to understand from other sources why they made such a decision.

What we have generally read here and there is that these restrictions mostly come from optimizing deployment and production readiness. But no one knows if this really is the case, or if these things are going to change in the near or far future.

---

### The Key Takeaway

For now, what matters is that we should **familiarize ourselves with Google ADK** (or any other framework we are working with) and study the documentation well—so we don't spend hours and hours figuring out a bug that does not appear clear at first glance.

With all the above said, let's move forward and fix the agent that we coded before.

## **Part 4. An Introduction to Multiagent Frameworks**

> Associated Agent: **biogeraphy_agent**

Alright, if we can't pass two tools of different types to a single agent, then the obvious alternative is to have **two agents that collaborate with each other**, where each uses one of the tools. That's exactly what we are going to do in this part, and that's our introduction to the concept of **multi-agent frameworks**.

---

### Why Multi-Agent Systems?

To be honest with you, even though an agent that can take many actions and use many tools on its own is a very exciting concept, a far more exciting concept for many developers are **multi-agent pipelines**. These pipelines leverage the power of more than one agent to achieve things that a single agent might not be able to do—or might not be able to do with good quality.

Now, some people look at multi-agent systems as very powerful workflows that can achieve superhuman results, advance science, and solve complex problems. Others look at them as an army of minions that may or may not be able to handle and automate some of the difficulties we have in our day-to-day life and practice.

Regardless of whichever camp you belong to, it is still good to understand the basics of how to connect two agents to each other.

---

### Three Main Multi-Agent Patterns in Google ADK

In the Google ADK framework, connection of agents to each other follows one of three main patterns:

**1. Sequential Pipeline:** As the name implies, a combination of two or more agents that are called one after the other, where the output of each may or may not be the input of the subsequent agents.

<img src="https://google.github.io/adk-docs/assets/sequential-agent.png" width="600" alt="Sequential Agent Pattern">

**2. Parallel Pipeline:** A setup where multiple agents are called to work on different aspects of the same problem at the same time. Notably, these aspects should be independent from each other—neither agent has access to the inputs or outputs of the other agents.

<img src="https://google.github.io/adk-docs/assets/parallel-agent.png" width="600" alt="Parallel Agent Pattern">

**3. Loop Agent:** An agent that keeps doing whatever it has been doing until a specific set of checkpoints have been met. You may wonder, "wasn't this supposed to be a multi-agent entity? Why are we talking about a single agent?" Because a loop agent usually acts as the **organizer** or **gatekeeper** of multiple agents working at the same time.

<img src="https://google.github.io/adk-docs/assets/loop-agent.png" width="600" alt="Loop Agent Pattern">

As we will see in the next examples, Google ADK enables you to wrap agents—and even multi-agent pipelines—under other agents or multi-agent pipelines. For example, imagine a sequential pipeline of agents that are all introduced as sub-agents to a loop agent, meaning that loop agent is going to keep calling this sequential pipeline until the desired outcome has been achieved.

---

### Solving Our Biography Agent Problem

Now, let us not confuse you. For this immediate problem that we have on our hands, the solution is very simple.

We are going to use a **SequentialAgent** from Google ADK that accepts a name and description as before, but unlike the prior agents we had, will **not receive direct instructions**. The reason is that even though this type of agent is named "sequential agent," it is not a real agent but mostly an agent whose task is to pass its inputs through a chain of sub-agents that it has.

Let's take a look at the root agent we can define for our biography agent:
```python
from google.adk.agents import SequentialAgent
from .sub_agents import research_agent, writing_agent

# Create sequential workflow: Research first, then write
root_agent = SequentialAgent(
    name="biography_agent",
    description=(
        "A sequential workflow that researches people using Google Search, "
        "then writes their biographies in markdown files. "
        "Uses separate agents for research and writing to avoid tool conflicts."
    ),
    sub_agents=[research_agent, writing_agent],
)
```

---

### Understanding Sub-Agents

As you can see here, we are passing two separate agents as sub-agents to this sequential agent. Now, you have already seen how the research agent and writing agent will operate *(feel free to check their source code again)*. Those two agents have nothing new—in fact, each of them is using a separate tool:

- The **research agent** uses the Google Search tool to look up a person
- The **writing agent** uses a markdown tool to write up the biography about that person

The combination of these is passed to this new sequential agent we have here.

**Important:** The order of the list you pass as `sub_agents` matters the most, because it defines which agent is going to receive the task first and then hand over to whom. In our case, the research agent does the first task and then the writing agent finishes it up.

---

### Code Organization Best Practices

Another important point to keep in mind: notice how we imported the agents in a relative manner from the `sub_agents` directory. This is another convention in Google ADK that matters a lot.

Whenever you are dealing with sub-agents and complicated multi-agent frameworks, it's better to rely as much as you can on the **hierarchy of files and folders** for your agents and sub-agents. Not only does Google ADK work best with this format, but it is also the most clear way of representing multi-agent pipelines in code.

As you will soon realize, these multi-agent pipelines tend to get confusing very quickly.

---

Enough talking! Take a look at the scripts of Part 4, and whenever you are ready, go ahead and try out our multi-agent pipeline in the next cell.

In [ ]:
# Run the Part 4 agent

chat_with_agent_clean('agents/part4/biography_agent', session_id='123')

Having fun with the agents? Cool! Don't forget to check the **markdown file** that the agentic framework created for you. You can find it under the `part4/biography_agent` folder.

Also, certainly take a look at the **session JSON file** that was saved. Do you see how much longer and more detailed this JSON file has become compared to our earlier agents?

---

### What's Inside the Session JSON?

Take a look at it—it contains:

- All the **input data** from you
- All the **output data** from the agents
- The **intermediate results** between agents
- All the **tool calls** that the agents made
- The **results** of those tool calls
- How the agents called the different tools they had in their arsenal
- All with their **token usage** and **timestamps**

---

Isn't that fascinating? If you were planning to deploy an agent in production, wouldn't you need something this detailed for monitoring, debugging, and optimization?

To us, this level of logging and transparency looks like one of the best aspects of Google ADK. It gives you full visibility into what your multi-agent system is doing at every step.

## **Part 5. Multiagentic Pipelines on Steroid**

> Associated Agent: **networking_agent**

# Building a Sophisticated Multi-Agent Pipeline

Great! Now that we are enjoying multi-agent pipelines, it is time to try a much more sophisticated pipeline so that you can see how we can merge and combine these different sequential, parallel, and loop workflows with each other.

For this task, we are not going to explore any new concepts, but we want to demonstrate how you can break a task into multiple subtasks and then delegate it to different agents.

---

## The Networking Task

The task that we want to handle in Part 5 might actually be useful for you. We call it a **networking task**.

The ultimate goal of this task is to:

1. Take someone's name (who might be a radiologist)
2. Pass it to a multi-agent pipeline that will:
   - First check if the person is actually a radiologist
   - If they are, find some of their most recent and most cited articles
   - Gather their background information
   - Find URLs that are publicly available about them on the internet (personal websites, social media, scientific portals, etc.)
3. At the very end, return the entire output as a nice markdown report

---

## Pipeline Architecture

Take a look at the following diagrams and see if you can understand how we have organized the agents to do this task:

### Architecture Diagram
```
┌─────────────────────────────────────────────────────────────────────────┐
│                         USER INPUT                                      │
│                    "Create a networking profile for [Name]"             │
└────────────────────────────┬────────────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                    ROOT AGENT (SequentialAgent)                         │
│                    networking_agent                                     │
└────────────────────────────┬────────────────────────────────────────────┘
                             │
                             │ Sequential Flow
                             │
                             ▼
┌───────────────────────────────────────────────────────────────────────────┐
│              STEP 1: Verification Agent (LlmAgent)                        │
│              radiologist_verification_agent                               │
│                                                                           │
│  Tools: [google_search]                                                   │
│                                                                           │
│  Input:  Person's name                                                    │
│  Output: Background information, verification status, professional details│
└────────────────────────────┬──────────────────────────────────────────────┘
                             │
                             │ Sequential Flow
                             │ (Output passed to next agent)
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────────┐
│         STEP 2: Parallel Research Agent (ParallelAgent)                 │
│         parallel_research_agent                                         │
│                                                                         │
│  ┌──────────────────────────────┐  ┌──────────────────────────────┐     │
│  │  URL Finder Agent            │  │  Article Agent               │     │
│  │  (LlmAgent)                  │  │  (LlmAgent)                  │     │
│  │                              │  │                              │     │
│  │  Tools: [google_search]      │  │  Tools: [get_semantic_       │     │
│  │                              │  │         scholar_papers]      │     │
│  │  Input: Person name +        │  │  Input: Person name +        │     │
│  │         background info      │  │         background info      │     │
│  │                              │  │                              │     │
│  │  Output: List of URLs        │  │  Output: Most recent &       │     │
│  │          (social media,      │  │          most cited papers   │     │
│  │           personal pages,    │  │          with metadata       │     │
│  │           institutional)     │  │                              │     │
│  └──────────────────────────────┘  └──────────────────────────────┘     │
│           │                                    │                        │
│           └──────────────┬─────────────────────┘                        │
│                          │ Parallel Execution                           │
│                          │ (Both agents run simultaneously)             │
│                          ▼                                              │
│              Combined Results: URLs + Publications                      │
└────────────────────────────┬────────────────────────────────────────────┘
                             │
                             │ Sequential Flow
                             │ (Combined results passed to formatter)
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────────┐
│              STEP 3: Formatter Agent (LlmAgent)                         │
│              profile_formatter_agent                                    │
│                                                                         │
│  Tools: [] (No tools - structured output only)                          │
│                                                                         │
│  Input:  All previous agent outputs:                                    │
│          - Background information (from verification_agent)             │
│          - URLs (from url_finder_agent)                                 │
│          - Publications (from article_agent)                            │
│                                                                         │
│  Output: NetworkingProfile (Pydantic model)                             │
│          - person_name: str                                             │
│          - background: str                                              │
│          - online_presence: List[URLInfo]                               │
│          - recent_publications: PublicationSection                      │
│            - most_recent_papers: List[Paper]                            │
│            - most_cited_papers: List[Paper]                             │
│          - contact_information: Optional[str]                           │
└────────────────────────────┬────────────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                         STRUCTURED OUTPUT                               │
│                    (NetworkingProfile Pydantic Model)                   │
└─────────────────────────────────────────────────────────────────────────┘
```

### Data Flow Diagram
```
User Input
    │
    ├─→ verification_agent
    │       │
    │       ├─→ Uses google_search tool
    │       │
    │       └─→ Output: Background info
    │               │
    │               ├─→ url_finder_agent (parallel)
    │               │       │
    │               │       ├─→ Uses google_search tool
    │               │       │
    │               │       └─→ Output: URLs
    │               │
    │               └─→ article_agent (parallel)
    │                       │
    │                       ├─→ Uses get_semantic_scholar_papers tool
    │                       │
    │                       └─→ Output: Publications
    │                               │
    │                               └─→ Combined Results
    │                                       │
    │                                       └─→ formatter_agent
    │                                               │
    │                                               └─→ Output: NetworkingProfile (structured)
```

---

Hopefully that was not too complicated! The main cornerstone of this setup is that we really want to **break up our tasks into as many smaller subtasks as possible** so that each subtask is easy enough for an agent to handle.

---

#### Why Break Tasks Into Smaller Pieces?

We really don't want to have an agent that is tasked with running many complicated processes. Here's why:

1. **Cost Efficiency:** You will need more expensive, larger language models for complex tasks
2. **Reliability:** Even with more powerful models, you will still increase the risk of failure

---

### Design Decisions Explained

#### Why Separate Verification and Research?

In this case, for example, you might argue that the verification agent and background information agent could be the same agent—and we agree! However, note that maybe some of the persons whose names we put in this pipeline are **not radiologists**.

If that's the case, we don't want the rest of the agentic workflow to be executed for those persons because we will be simply **spending tokens on something that is never going to be useful** for us.

#### Why Use Semantic Scholar API Instead of Google Search?

You might notice that the article agent is using a relatively sophisticated `get_semantic_scholar_papers` tool even though we could have relied on the Google Search tool for finding papers for someone.

That certainly would work, but it's not going to be as accurate as the Semantic Scholar API. However, as we will finally see, the Semantic Scholar API also has its own pitfalls.

---

#### Let's Test It!

Regardless, let's go ahead and run this pipeline and talk afterwards.

**Do you have someone in mind to stalk?** Now's the time to give it a try! 😄

**Note**:<br>
The Networking agent chatbot works more reliably if you have a Semantic Scholar API key. This key is free and can be obtained through the following link. If you have a key, uncomment the cell below and set it in your notebook.

In [ ]:
# os.environ['SEMANTIC_SCHOLAR_API_KEY'] = 'YOUR_SEMANTIC_SCHOLAR_API_KEY'

In [ ]:
chat_with_agent_clean('agents/part5/networking_agent', session_id='123')

Wow, that was a lot of work that these agents did for us, wasn't it?

Well, we agree! Despite this impressive work, there are still a few points we need to discuss.

---

### The Nature of Multi-Agent Workflows

Did you notice that in these past two agents (Parts 4 and 5), we actually spent **less time talking** with our agents and let our agents work more and more **on their own**?

Well, that's what real multi-agent workflows look like! You **rarely use these multi-agent workflows for chatting**. Mostly, you let them go into the wilderness and do jobs for you that might not be easy for you to do—or at least would take a lot of your time.

Even though multi-agent pipelines might still have some requirement to chat with the user, the chat assignment is often **delegated to a specific one of the many agents** that are present in the pipeline, and is mostly used for cases where the agent runs into any difficulty.

**Key Takeaway:** We would like you to have this concept in mind—most of our day-to-day interactions with agents are for conversations, but a **real agentic workflow is often focused on doing a real task**.

---

### JSON Output Instead of Markdown

The other thing you might have noticed is that the output of the multi-agent pipeline above is **not the markdown file** that we wanted, but a **JSON file**. Well, why did this happen?

This, to be honest with you, wasn't designed on purpose. Even though you could try to ask the formatter agent (which is the last agent in our pipeline) to give you back a markdown file and maybe save it to disk as the biography agent did, you might notice that this is **not a very reliable setup**.

Because the pipeline has gotten too complicated now, chances are very high that the formatter agent might forget to call its tool or might not even return the markdown file at all. If you have doubts about it, go ahead and change the instructions for that agent!

---
The above discussion brings up two more questions:

### 1. How did the agent decide to give us a JSON file?

If this agent was not tasked to bring markdown, then how did it decide to give us a JSON file?

### 2. How can we get the markdown version?

What can we do to get our hands on the markdown version of this information?

---

## Answer: Structured Outputs

To answer the first question, we should tell you that another very useful feature of Google ADK agents is that you can **enforce them to follow a structured format for their outputs**.

In almost all cases, a structured output is enforced using **JSON schemas** or **Pydantic classes**. In both cases, the output is going to be a JSON file.

**Why JSON?**
- JSON files are very frequently used in agents because they are the standard in web programming and JavaScript
- They are the bread and butter of API building and passing data between different APIs
- Developers love them for structured data exchange

---

## How We Enforced Structured Output

Let's take a look at how we actually enforced the formatter agent to create this JSON for us:
```python
from pydantic import BaseModel
from typing import Optional, List
from google.adk.agents import LlmAgent

# Define Pydantic models for structured output
class Paper(BaseModel):
    title: str
    journal: str
    year: int
    url: str
    citations: int
    authors: str


class PublicationSection(BaseModel):
    most_recent_papers: List[Paper]
    most_cited_papers: List[Paper]


class URLInfo(BaseModel):
    category: str
    url: str
    description: str
    platform: str


class NetworkingProfile(BaseModel):
    person_name: str
    background: str
    online_presence: List[URLInfo]
    recent_publications: PublicationSection
    contact_information: Optional[str] = None


# Define the formatter agent - Step 3: Compile everything into structured output
formatter_agent = LlmAgent(
    name="profile_formatter_agent",
    model="gemini-2.5-flash",
    output_schema=NetworkingProfile,  # ← This enforces the structured output!
    instruction=(
        "You are a profile formatting agent. Your role is to compile all gathered information "
        "into a structured networking profile.\n\n"
        
        "When given information from previous agents:\n"
        "1. Review all information provided:\n"
        "   - Background information from verification agent\n"
        "   - URLs from URL finder agent (social media, personal web pages, etc.)\n"
        "   - Recent articles from article agent (including Most Recent Papers and Most Cited Papers sections)\n\n"
        "2. Extract and structure the information according to the required format:\n"
        "   - person_name: The full name of the person\n"
        "   - background: Background information from verification agent (clear, professional format)\n"
        "   - online_presence: List of URLInfo objects with:\n"
        "     * category: Type (e.g., \"Social Media\", \"Personal Website\", \"Institutional\")\n"
        "     * url: The direct URL\n"
        "     * description: Brief description of the page\n"
        "     * platform: Platform or website name (e.g., \"LinkedIn\", \"Twitter\", institution name)\n"
        "   - recent_publications: PublicationSection with:\n"
        "     * most_recent_papers: List of Paper objects (up to 10)\n"
        "     * most_cited_papers: List of Paper objects (up to 10)\n"
        "     Each Paper should have: title, journal, year (integer), url, citations (integer), authors\n"
        "   - contact_information: Contact info if available from background research (optional)\n\n"
        "3. Structure the response according to the NetworkingProfile schema.\n"
        "4. Ensure all data is accurate and well-organized.\n"
        "5. Return the structured profile - this will be returned to the user.\n\n"
        "IMPORTANT: Return the structured profile according to the NetworkingProfile schema. "
        "Do NOT save any files - just return the structured data."
    ),
    description=(
        "Compiles all gathered information (background, URLs, articles) into a "
        "structured networking profile using Pydantic models."
    ),
    tools=[],
)
```

---

**The key line:** `output_schema=NetworkingProfile`

This parameter tells the agent that its output **must** conform to the `NetworkingProfile` Pydantic model. The agent will automatically structure its response as a JSON object matching this schema, ensuring consistency and making the data easy to work with programmatically.

### Issue 2: Converting JSON to Markdown: A Better Approach

Alright, now let's go back to our second question: **How can we convert this JSON output to a markdown format?**

---

### The Problem with Terminal-Based Interactions

Well, in real-life programming, you cannot rely on interacting with your agents in a terminal. Even though we created a beautiful UI to avoid seeing those ugly terminal interactions, this still has a lot of drawbacks.

Most important of which is that if we do not rely on saving files to disk, we have **no way of getting the output of our agents** unless by running some very ugly bash commands.

We are sure you agree that this is not an easy way to interact with the outputs of our agents in our notebook environment.

---

## Time to Go Deeper: Running Agents Programmatically

So it's finally time to learn **how to run an agent or a multi-agent pipeline from within our code** without relying on the Google ADK run commands.

In fact, it's time to learn that the `adk run` command is just a **wrapper** around maybe a handful of lines of code that we can write ourselves—and then get **full control** over our agent pipeline.

To do this, we need to familiarize ourselves with three key concepts:

### Application (App)

**App** is the name of the application that we are creating. Google ADK sometimes imposes limitations on the name of the apps, but for now, let's imagine that you can name your application whatever you like.

### Session

**Session** is basically a separate instance of conversation or activity that you assign to your agent pipeline. Each user interacting with the agent pipeline will have their own session, and the sessions are separated and fully independent from each other.

Think of the session as a **context** that you can engineer for the agent and fill it with data, information, and insights that will help your agents work as expected. You might have heard about things like **memory** that agents could have and can rely on for remembering things—well, these are things that someone can save on an agent pipeline session and are immediately accessible to agents.

Regardless, let's put aside the deeper aspects of sessions as they go a little bit beyond the scope of our today's workshop.

### Session Service

**SessionService** is responsible for managing conversation history and state for different users and sessions. It keeps track of the messages exchanged.

As we were talking about sessions, those of you who might have more experience with coding could have imagined: **where does all this information get stored?** Do we store this in memory or can we store them in databases?

Well, the answer is yes to both! The default behavior of Google ADK is to save the sessions in **the system's memory** using `InMemorySessionService`, which is suitable for testing and simple applications. But you can also configure it to save sessions in **SQLite databases**, which is great for production. Even more interesting, you can save everything on **Google Cloud services**, which offer specific types of databases for enterprise-scale deployments.

### Runner

**Runner** is the main engine to run your entire pipeline and all the sessions at the same time. It orchestrates the interaction flow—it takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the SessionService, and yields events representing the progress of the interaction. The runner keeps track of all the sessions and ensures the multi-agent pipeline can adequately reply and respond to all the requests it is getting from different sessions.

---

## Running Our Networking Agent Programmatically

For now, we are going to define a **runner and session** for our networking agent and see how we can get the output of the multi-agent pipeline and save it as markdown.

Note that previously the `adk run` command was handling all the outputs from the agents for us and could simply pass the ultimate response of the agent pipeline back to be seen by us. But when we are dealing with the runner and sessions ourselves, we should also be **responsible for parsing the outputs** of the agents and figuring out which responses are intermediate and which responses are final.

This is exactly what we will do in the next couple of cells. Let's go through all these codes and see if you can follow them.

In [ ]:
# Custom Runner Implementation for Networking Agent
# This demonstrates how to build custom runners and sessions
# Note: In Colab, you'll need to upload the scripts directory first

import json
import uuid
import asyncio
from pathlib import Path

# Import the networking agent and its components
from agents.part5.networking_agent.agent import root_agent
from agents.part5.networking_agent.sub_agents.formatter_agent import NetworkingProfile
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.runners import types

# Run this in a Colab cell
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# @title A utility tool that will help us convert the JSON output to a Markdown file.

def profile_to_markdown(profile: NetworkingProfile) -> str:
    """Convert NetworkingProfile Pydantic model to markdown format."""
    markdown_parts = []

    # Header
    markdown_parts.append(f"# {profile.person_name} - Networking Profile\n")

    # Background
    markdown_parts.append("## Background\n")
    markdown_parts.append(f"{profile.background}\n")

    # Online Presence
    markdown_parts.append("## Online Presence\n")
    if profile.online_presence:
        # Group URLs by category
        categories = {}
        for url_info in profile.online_presence:
            category = url_info.category
            if category not in categories:
                categories[category] = []
            categories[category].append(url_info)

        # Write each category
        for category, urls in categories.items():
            markdown_parts.append(f"### {category}\n")
            for url_info in urls:
                markdown_parts.append(f"- [{url_info.platform}]({url_info.url}) - {url_info.description}\n")
    else:
        markdown_parts.append("No online presence information available.\n")
    markdown_parts.append("")

    # Recent Publications
    markdown_parts.append("## Recent Publications\n")

    # Most Recent Papers
    if profile.recent_publications.most_recent_papers:
        markdown_parts.append("### Most Recent Papers\n")
        for paper in profile.recent_publications.most_recent_papers:
            markdown_parts.append(
                f"- **{paper.title}** - *{paper.journal}* ({paper.year}) "
                f"[Link]({paper.url}) - Citations: {paper.citations}\n"
            )
        markdown_parts.append("")

    # Most Cited Papers
    if profile.recent_publications.most_cited_papers:
        markdown_parts.append("### Most Cited Papers\n")
        for paper in profile.recent_publications.most_cited_papers:
            markdown_parts.append(
                f"- **{paper.title}** - *{paper.journal}* ({paper.year}) "
                f"[Link]({paper.url}) - Citations: {paper.citations}\n"
            )
        markdown_parts.append("")

    # Contact Information
    if profile.contact_information:
        markdown_parts.append("## Contact Information\n")
        markdown_parts.append(f"{profile.contact_information}\n")

    return "".join(markdown_parts)

In [ ]:
# @title Main function for creating runner, session, and executing the agents.

async def run_agent_async(person_name: str) -> NetworkingProfile:
    """Run the networking agent with a given person's name (async version)."""
    # Create session service
    session_service = InMemorySessionService()

    # Create a unique session ID
    session_id = str(uuid.uuid4())
    user_id = "user"
    app_name = "agents"

    # Create the session explicitly before running (as per ADK documentation)
    _ = await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id
    )

    # Create runner with the root agent
    runner = Runner(
        agent=root_agent,
        app_name=app_name,
        session_service=session_service
    )

    # Create the message content
    message = types.Content(
        parts=[types.Part(text=f"Create a networking profile for {person_name}")],
        role="user"
    )

    # Run the agent and collect events using run_async
    events = []
    final_response_text = None

    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session_id,
            new_message=message
        ):
            events.append(event)

            # Check if this is the final response (as per ADK documentation)
            if hasattr(event, 'is_final_response') and event.is_final_response():
                if hasattr(event, 'content') and event.content:
                    if hasattr(event.content, 'parts') and event.content.parts:
                        for part in event.content.parts:
                            if hasattr(part, 'text') and part.text:
                                final_response_text = part.text
                                # Try to parse as JSON (structured output)
                                try:
                                    data = json.loads(part.text)
                                    if isinstance(data, dict) and 'person_name' in data:
                                        return NetworkingProfile(**data)
                                except (json.JSONDecodeError, TypeError, ValueError):
                                    pass

            # Also check event.content for structured output
            if hasattr(event, 'content') and event.content:
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            # Try to parse as JSON (structured output)
                            try:
                                data = json.loads(part.text)
                                if isinstance(data, dict) and 'person_name' in data:
                                    return NetworkingProfile(**data)
                            except (json.JSONDecodeError, TypeError, ValueError):
                                pass
    except Exception as e:
        print(f"Error during event processing: {e}")
        import traceback
        traceback.print_exc()

    # If we have a final response text but couldn't parse it, try one more time
    if final_response_text:
        try:
            data = json.loads(final_response_text)
            if isinstance(data, dict) and 'person_name' in data:
                return NetworkingProfile(**data)
        except (json.JSONDecodeError, TypeError, ValueError):
            pass

    # Debug: Print event types to help diagnose
    if events:
        print(f"Debug: Received {len(events)} events")
        print(f"Debug: Event types: {[type(e).__name__ for e in events[-5:]]}")
        if final_response_text:
            print(f"Debug: Final response text (first 500 chars): {final_response_text[:500]}")

    raise ValueError(f"Could not parse NetworkingProfile from agent response. "
                    f"Received {len(events)} events.")


In [ ]:
person_name = "Pouria Rouzrokh"  # Change this to any radiologist's name

print(f"Researching networking profile for: {person_name}")
print("This may take a few moments...\n")

try:
    # Run the agent
    profile = asyncio.run(run_agent_async(person_name))

    # Convert to markdown
    markdown_content = profile_to_markdown(profile)

    # Display the markdown (in Colab, this will render nicely)
    from IPython.display import Markdown, display
    display(Markdown(markdown_content))

    # Optionally save to file
    # output_path = Path("/content") / f"{person_name.replace(' ', '_').lower()}_profile.md"
    # with open(output_path, 'w', encoding='utf-8') as f:
    #     f.write(markdown_content)
    # print(f"\n✓ Profile saved to: {output_path}")

except Exception as e:
    print(f"\n✗ Error running agent: {e}")
    import traceback
    traceback.print_exc()

Researching networking profile for: Pouria Rouzrokh
This may take a few moments...



ERROR:asyncio:Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7b33099af7d0>, 584.051249309)])']
connector: <aiohttp.connector.TCPConnector object at 0x7b330982b080>


# Pouria Rouzrokh - Networking Profile
## Background
Pouria Rouzrokh, MD, MPH, MHPE, is a Diagnostic Radiology Resident at Yale University, where he engages in both clinical practice and research focusing on the intersection of medicine and technology. He also holds an Adjunct Assistant Professorship in Radiology at Mayo Clinic. Education and Training: Doctor of Medicine (MD): Tehran University of Medical Sciences, Tehran, Iran (2009-2018). His dissertation focused on developing, implementing, and evaluating a curriculum for teaching medical students and residents communication skills in difficult situations. Master of Public Health (MPH): Tehran University of Medical Sciences, Tehran, Iran (2012-2016). His dissertation focused on a gamified curriculum for teaching medical students about social determinants of health. Master of Health Professions' Education (MHPE): Tehran University of Medical Sciences, Tehran, Iran (2011-2014). Postdoctoral Research Fellowship: Mayo Clinic, Rochester, MN, USA (2020-2024), with research areas in artificial intelligence, machine learning, and radiology. Internal Medicine Internship: Griffin Hospital, Derby, CT, USA (2024-2025). Diagnostic Radiology Residency: Yale University School of Medicine, New Haven, CT, USA (2025-2030). Research Interests and Specialties: Dr. Rouzrokh is a physician-scientist and developer specializing in the application of artificial intelligence (AI) and machine learning (ML) in healthcare, particularly in medical imaging. His work involves developing AI tools for automating quantitative measurements in radiology, curating large-scale datasets, building systematic imaging registries, and optimizing clinical workflows. He is also passionate about the explainability and fairness of AI models, having investigated algorithmic bias in medical AI. His contributions span musculoskeletal imaging, neuroimaging, and general clinical data science. He is involved in research initiatives focused on developing and promoting AI tools for advancing radiology and medical imaging, and has experience as a data scientist for orthopedic surgery AI tools. Professional Achievements and Activities: Recipient of the 2023 NAIRS Annual Research Award. Member of the Radiological Society of North America (RSNA) (2020 - Present). Member of the Society for Imaging Informatics in Medicine (SIIM) (2020 - Present). Reviewer for Radiology Journal (2024 - Present) and Radiology: Artificial Intelligence Journal (2023 - Present). Editorial Board Member for RadioGraphics Journal (Trainee Editorial Board - RG-Team) (2023 - 2025). Co-founder of Vision Research Lab, a research initiative for AI tools in radiology. Authored numerous publications on AI in radiology and medical imaging. Recognized with honors such as the University Chancellor Award (Tehran University of Medical Sciences, 2018), Runner-Up Award for Innovation in Medical Education (Ministry of Health, Iran, 2018), Third Place National Award for Innovation in Medical Education (Ministry of Health, Iran, 2017), and a Gold Medal in the National Medical Student Olympiad (Ministry of Health, Iran, 2015).
## Online Presence
### Personal Website
- [PouriaRouzrokh.com](https://www.pouriarouzrokh.com) - Official website showcasing his work as an AI researcher, developer, and physician-scientist, including his education, research, and professional activities.
### Institutional Profile
- [Yale University School of Medicine](https://medicine.yale.edu/profile/pouria-rouzrokh/) - Profile page on the Yale School of Medicine website, detailing his diagnostic radiology residency, research interests, and biography.
- [Mayo Clinic](https://www.mayoclinic.org/research/labs/radiology-informatics-bradley-j-erickson/faculty-staff) - Listing as an Assistant Professor of Radiology within the Radiology Informatics lab at Mayo Clinic Research.
- [Griffin Hospital](https://meded.griffinhealth.org/residents) - Listing on the Griffin Hospital medical education residents page, showing his PGY-2 status.
### Social Media
- [LinkedIn](https://www.linkedin.com/in/pouria-rouzrokh) - Professional networking profile on LinkedIn.
- [X (formerly Twitter)](https://x.com/PRouzrokh) - Microblogging and social networking profile on X (formerly Twitter).
### Research Profile
- [Google Scholar](https://scholar.google.com/citations?user=yWv1N-wAAAAJ&hl=en) - Google Scholar profile listing his academic publications, citations, and h-index.
- [ResearchGate](https://www.researchgate.net/profile/Pouria-Rouzrokh) - ResearchGate profile for academic networking and sharing research, including publications and collaborations.
### Developer Profile
- [GitHub](https://github.com/PouriaRouzrokh) - GitHub profile showcasing his projects and code repositories as a physician and AI developer.
### Professional Publication
- [RSNA Journals](https://pubs.rsna.org/doi/full/10.1148/ryai.220059) - An article titled 'Mitigating Bias in Radiology Machine Learning: 1. Data Handling' authored by Pouria Rouzrokh on RSNA's Radiology: Artificial Intelligence journal.
## Recent Publications
### Most Recent Papers
- **A Current Review of Generative AI in Medicine: Core Concepts, Applications, and Current Limitations** - *Current Reviews in Musculoskeletal Medicine* (2025) [Link](https://www.semanticscholar.org/paper/5b373d8565443bcb806fe2ed6cd7185d45c2bca2) - Citations: 7
- **Challenges of Gingival Surgery Approaches in the Treatment of Peri-Implantitis: A Systematic Review** - *Journal of Pharmacy and Bioallied Sciences* (2025) [Link](https://www.semanticscholar.org/paper/27018b14939b91ca0e263b7f66a694be854ca2dd) - Citations: 0
- **LatteReview: A Multi-Agent Framework for Systematic Review Automation Using Large Language Models** - *arXiv.org* (2025) [Link](https://www.semanticscholar.org/paper/d0e126289ff34c517668da35a9a205012dd8bdec) - Citations: 0
- **Applying for an American radiology residency as an international medical graduate: The NRMP match.** - *Current problems in diagnostic radiology* (2025) [Link](https://www.semanticscholar.org/paper/9350ba6502fa73c6866ad044f3e701cde02c2961) - Citations: 0
- **Winning at the Radiology Podium: First-timer's Guide to Crafting and Delivering a Memorable Conference Presentation.** - *Radiographics* (2025) [Link](https://www.semanticscholar.org/paper/05557d6b1851f113192ff7094a493e51c1d7a02e) - Citations: 0
- **Bone Appetit: Skellytour Sets the Table for Robust Skeletal Segmentation.** - *Radiology: Artificial Intelligence* (2025) [Link](https://www.semanticscholar.org/paper/369ced05ec8782d9224e4687d3753b1ba93cce78) - Citations: 0
- **The Path to Becoming a Peer Reviewer for Radiology Journals: Recipe for Trainees.** - *Radiographics* (2025) [Link](https://www.semanticscholar.org/paper/3e9ca1d583080caaf68bc075ec78864f98c77846) - Citations: 0
- **Diagnostic performance of X-ray-based deep learning models for detecting ankle and foot fractures: a systematic review and meta-analysis.** - *Skeletal Radiology* (2025) [Link](https://www.semanticscholar.org/paper/5eebeaebbbee374978972fcc9c72b48f6441719d) - Citations: 0
- **Explaining explainability: The role of XAI in medical imaging.** - *European Journal of Radiology* (2024) [Link](https://www.semanticscholar.org/paper/68adc9f9b87fb0b9100d5698505b6b58f268e074) - Citations: 9
- **CONFLARE: CONFormal LArge language model REtrieval** - *arXiv.org* (2024) [Link](https://www.semanticscholar.org/paper/02b1b4594a79dafe57ac3411cda5e83c35e22b91) - Citations: 4
### Most Cited Papers
- **Mitigating Bias in Radiology Machine Learning: 1. Data Handling.** - *Radiology: Artificial Intelligence* (2022) [Link](https://www.semanticscholar.org/paper/caaf3277faea39df66b92d950b3a5b406f1d2310) - Citations: 101
- **THA-AID: Deep Learning Tool for Total Hip Arthroplasty Automatic Implant Detection with Uncertainty and Outlier Quantification.** - *Journal of Arthroplasty* (2023) [Link](https://www.semanticscholar.org/paper/73de9eba5ba25581ad8666147a86a4bc2ec4baf4) - Citations: 20
- **Deep Learning Dramatically Reduces the Work Associated with Image Cataloguing and Analysis: Commentary on an article by Pouria Rouzrokh, MD, MPH, MHPE, et al.: "Applying Deep Learning to Establish a Total Hip Arthroplasty Radiography Registry. A Stepwise Approach".** - *Journal of Bone and Joint Surgery. American volume* (2022) [Link](https://www.semanticscholar.org/paper/78203686aacd193766a4229598f4f9c2e5b78554) - Citations: 9
- **Explaining explainability: The role of XAI in medical imaging.** - *European Journal of Radiology* (2024) [Link](https://www.semanticscholar.org/paper/68adc9f9b87fb0b9100d5698505b6b58f268e074) - Citations: 9
- **A Current Review of Generative AI in Medicine: Core Concepts, Applications, and Current Limitations** - *Current Reviews in Musculoskeletal Medicine* (2025) [Link](https://www.semanticscholar.org/paper/5b373d8565443bcb806fe2ed6cd7185d45c2bca2) - Citations: 7
- **CONFLARE: CONFormal LArge language model REtrieval** - *arXiv.org* (2024) [Link](https://www.semanticscholar.org/paper/02b1b4594a79dafe57ac3411cda5e83c35e22b91) - Citations: 4
- **Preparing Radiologists for an Artificial Intelligence-enhanced Future: Tips for Trainees.** - *Radiographics* (2024) [Link](https://www.semanticscholar.org/paper/c73f29ad52cf271b46e4abf5eba63a40d0d177e1) - Citations: 3
- **RadRotator: 3D Rotation of Radiographs with Diffusion Models** - *arXiv.org* (2024) [Link](https://www.semanticscholar.org/paper/212a1939b0b69c3ef3d55413da7bf369438337ee) - Citations: 2
- **Ovarian Fibromatosis.** - *Radiographics* (2024) [Link](https://www.semanticscholar.org/paper/068233c9def01293c095ee4eee3925d0eb24dcec) - Citations: 1
- **The Era of Artificial Intelligence in Radiology: How to Prepare for a Different Future.** - *Academic Radiology* (2024) [Link](https://www.semanticscholar.org/paper/c7598f5e66450b302cdfe7591aa20c71c2ea1c86) - Citations: 1
## Contact Information
rouzrokh.pouria@mayo.edu


# Final Words

All right, now that you saw a very complicated multi-agent pipeline, we think it's time to adjourn this notebook. However, there is much, much more content to learn about agentic AI and the Google ADK framework that we couldn't cover today.

---

## Additional Resources

The following are a list of resources that you can visit yourself and learn more about:

- **[Agent Development Kit Documentation](https://google.github.io/adk-docs/)**
- **[ADK Masterclass](https://www.youtube.com/watch?v=P4VFL9nIaIA])**
- **[ADK Tutorial Google Colab by Google](https://colab.research.google.com/github/google/adk-docs/blob/main/examples/python/tutorial/agent_team/adk_tutorial.ipynb)**

---

## Assignment: Part 6 - Radiology Guidelines Research Agent

Also, it is worth mentioning that there is a **Part 6** to this notebook that we did not cover. It contains an entirely new multi-agent pipeline for researching radiology guidelines.

We believe playing with this new agent framework and maybe trying to put it to work is a good assignment for you. We did not include the codes to run that part here, but it should not be that difficult for you to run the Part 6 multi-agent framework by looking at and reflecting on the content that we have covered so far.

---

At the very end, we would like to thank you for reading, listening, or watching us until the very end. We wish you can create multi-agent pipelines that achieve things that you yourself could never do alone.

**Thank you!** 🚀